In [1]:
import AdventOfCode

In [2]:
input <- dayString 20

In [3]:
data Path a = Path [Segment a]
    deriving (Eq, Show)

data Segment a
    = Leaf a
    | Node [Path a]
    deriving (Eq, Show)

In [4]:
{-# LANGUAGE ScopedTypeVariables #-}
import Data.List

parseLeaf str = let
    (t, rest) = span (`elem` "NEWS") str
    in (Leaf t, rest)

parseNode acc ('(':str) = let
    (t, rest) = parsePath [] str
    in parseNode (acc++[t]) rest
parseNode acc ('|':str) = let
    (t, rest) = parsePath [] str
    in parseNode (acc++[t]) rest
parseNode acc (')':str) = (Node acc, str)

parsePath :: [Segment String] -> String -> (Path String, String)
parsePath acc []  = (Path acc, [])
parsePath acc str@(x:xs) = case x of
    '(' -> let
        (group, str') = parseNode [] str
        in parsePath (acc++[group]) str'
    '|' -> (Path acc, str)
    ')' -> (Path acc, str)
    _ -> let
        (chunk, str') = parseLeaf str
        in parsePath (acc++[chunk]) str'

parsePath [] "NSW(EE|NN|)S"

(Path [Leaf "NSW",Node [Path [Leaf "EE"],Path [Leaf "NN"],Path []],Leaf "S"],"")

In [5]:
trimmed = init $ tail input
processed = fst $ parsePath [] trimmed

In [6]:
maxPath (Path ts) = sum $ map maxSegment ts
maxSegment (Leaf str) = length str
maxSegment (Node rs) = let
    subpaths = map maxPath rs
    in if 0 `elem` subpaths then 0 else maximum subpaths

maxPath processed

3958

In [7]:
{-# LANGUAGE BangPatterns #-}

import qualified Data.Map.Strict as Map

move :: (Int, Int) -> Char -> (Int, Int)
move (x,y) c = case c of
    'N' -> (x, y+1)
    'W' -> (x-1, y)
    'E' -> (x+1, y)
    'S' -> (x, y-1)
    
visitString
    :: (Map.Map (Int, Int) Int)
    -> (Int, Int)
    -> Int
    -> String
    -> (Map.Map (Int, Int) Int, (Int, Int), Int)
visitString rooms curPos doorCount str = case str of
    [] -> (rooms, curPos, doorCount)
    _ -> update rooms curPos doorCount str
    where
        update rs pos count s = case s of
            [] -> (rs, pos, count)
            x:xs -> let
                pos' = move pos x
                count' = count + 1
                !rs' = Map.insertWith min pos' count' rs
                in update rs' pos' count' xs

In [8]:
visitGraph :: Path String -> [(Int, Int)] -> (Int, Int) -> Map.Map (Int, Int) Int -> Int -> ([(Int, Int)], (Int, Int), Map.Map (Int, Int) Int, Int)
visitGraph (Path []) posStack curPos rooms doorCount =
    (posStack, curPos, rooms, doorCount)
visitGraph (Path (Leaf str:ps)) posStack curPos rooms doorCount =
    let (rooms', curPos', doorCount') = visitString rooms curPos doorCount str
    in visitGraph (Path ps) posStack curPos' rooms' doorCount'
visitGraph (Path (Node ns:ps)) posStack curPos rooms doorCount =
    let (curPos':posStack', rooms') = visitNode (curPos:posStack) rooms doorCount ns
    in visitGraph (Path ps) posStack' curPos' rooms' doorCount
    
visitNode :: [(Int, Int)] -> Map.Map (Int, Int) Int -> Int -> [Path String] -> ([(Int, Int)], Map.Map (Int, Int) Int)
visitNode         posStack rooms  _         [] = (posStack, rooms)
visitNode (curPos:posStack) rooms doorCount (x:xs) = let
    (posStack', curPos', rooms', doorCount') = visitGraph x posStack curPos rooms doorCount
    in visitNode (curPos:posStack) rooms' doorCount xs

visited = visitGraph processed [] (0,0) Map.empty 0

In [9]:
(_, _, rooms, _) = visited

length $ filter (>=1000) $ Map.elems rooms

8566